# **Clase 22 - Large Language Models**

MDS7202: Laboratorio de Programación Científica para Ciencia de Datos

## **Objetivos**

- Conocer qué son los LLM y cómo trabajarlos con LangChain
- Aprender a generar chains de prompts
- Comprender el paradigma de Agentes
- Entender como generar una solución RAG
- Introducir al estudiante a soluciones multiagente

## **Configuración Inicial 🧐**

Para esta clase necesitaremos configurar las credenciales de algunos servicios a utilizar, en específico:

### **Google AI Studio**

Usaremos `Google AI Studio` para habilitar el uso de LLMs y Embeddings de Google. Simplemente deben registrarse con su cuenta google y obtener su API KEY desde el siguiente enlace: [Google AI Studio](https://aistudio.google.com/app/u/1/apikey).

### **Tavily**

En paralelo, utilizaremos `Tavily` como motor de búsqueda para potenciar las respuestas de nuestros agentes. Tal como en el paso anterior, solo deben registrarse y obtener su API KEY desde el siguiente enlace: [Tavily](https://tavily.com/).

### **Configurar credenciales en ambiente**

Una vez se tienen todas las credenciales, pasamos a activarlas en nuestro ambiente local por medio del siguiente código:

In [ ]:
import getpass
import os

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google AI API key: ")

if "TAVILY_API_KEY" not in os.environ:
    os.environ["TAVILY_API_KEY"] = getpass.getpass("Enter your Tavily API key: ")

O si les sale más fácil, también pueden hacerlo a través de un archivo **.env** (esto funciona mejor cuando trabajan desde sus máquinas locales).

Sólo debemos crearlo y escribir en él todas las credenciales:

```python
GOOGLE_API_KEY="<YOUR_GOOGLE_API_KEY>"
TAVILY_API_KEY="<YOUR_TAVILY_API_KEY>”
```

Luego, cargamos las credenciales al ambiente:

In [ ]:
%pip install --upgrade --quiet python-dotenv

In [ ]:
from dotenv import load_dotenv

load_dotenv() # cargar las variables guardadas en el archivo .env

## **¿Qué son los Large Language Models? 🤔**

Los **Large Language Models (LLM)** son modelos de lenguaje basados en **redes neuronales del tipo transformer** entrenados con una gran cantidad de texto. Estos modelos poseen las siguientes características distintivas:

- Son modelos **entrenados con grandes volúmenes de lenguaje** (corpus del tamaño de internet), permitiéndoles aprender patrones complejos del lenguaje y obtener una comprensión semántica profunda.

- Son modelos con una **gran cantidad de parámetros** (miles de millones!), lo que les permite captar sutilezas lingüísticas, estilo y tono en un nivel sin precedentes.

Adicionalmente, existen 2 grandes tipos de LLM:

- Modelos basados en **representaciones**: Son modelos orientados a **aprender representaciones** (o *embeddings*) del lenguaje. Un ejemplo notable de este tipo de modelos es [BERT](https://arxiv.org/abs/1810.04805).
- Modelos basados en **completion**: Son modelos diseñados para **predecir la siguiente palabra** de un texto. Un ejemplo notable son los modelos GPT (Generative Pre-trained Transformers), como GPT3.5 o GPT4.

De estos modelos, son los GPT los que han obtenido más fama en el último tiempo, dando pie a una gran cantidad de soluciones basadas en esta tecnología.
Como podrán suponer, para esta clase nos concentraremos en los LLM de completion.

> **Pregunta**: ¿Por qué creen que que los GPT basados en completion tuvo mayor éxito?

<center>
<img src='https://media4.giphy.com/media/qAtZM2gvjWhPjmclZE/giphy.gif' width=450  />
</center>



## **LangChain 🦜**

`LangChain` es un framework de código abierto diseñado para desarrollar aplicaciones basadas en LLM. Basa su funcionamiento en los siguientes componentes:

<center>
<img src='https://raw.githubusercontent.com/MDS7202/MDS7202/main/recursos/2024-01//LLM/langchain.png' width=450  />
</center>

**1. Prompts y Plantillas de Prompts**: Los prompts son consultas que enviamos a los LLMs, y su calidad impacta las respuestas. LangChain facilita la creación y gestión de prompts mediante plantillas que combinan instrucciones, contenido y consultas.

```python
template = """
You are required to answer the following question in form of bullet points based on the provided context.
The answer should be answer as a doctor and your language is spanish.:
{context}
Now based on above context answer the following question:
{question}

Answer:
"""
```

**2. Modelos**: LangChain no incluye LLMs, pero permite integrar fácilmente varios modelos de lenguaje (como GPT-3, BLOOM), de chat (como GPT-3.5-turbo) y de embedding de texto (de CohereAI, HuggingFace y OpenAI) mediante un simple framework donde solo necesitas la API Key o cargar el modelo en memoria.

```python
from langchain.llms import OpenAI

openai = OpenAI(
   openai_api_key=”YOUR OPEN AI API KEY”,
   model_name="gpt-3.5-turbo-16k",
)
```

**3. Chains**: LangChain permite crear flujos de trabajo, o "chains," que son secuencias de llamadas a modelos de lenguaje o a herramientas externas. Estos flujos pueden incluir varias etapas de procesamiento, donde la salida de una etapa se convierte en la entrada de la siguiente. son el simil de pipelines de `scikit-learn`.

```python
chain = prompt | llm | StrOutputParser()
```

**4. Memoria**: Por defecto, las cadenas en LangChain son sin estado, es decir, no guarda un registro de las interacciones anteriores hechas con el modelo. Para superar esto, LangChain nos asiste con *buffers* para almacenar de forma iterativa las interacciones realizadas.

<center>
<img src='https://raw.githubusercontent.com/MDS7202/MDS7202/main/recursos/2024-01//LLM/memory.png' width=600 />
</center>

**5. Agentes**: Los agentes en LangChain toman decisiones sobre acciones según la entrada o el estado del flujo, como buscar información o llamar a una API, lo que permite crear aplicaciones interactivas y adaptables.

<center>
<img src='https://raw.githubusercontent.com/MDS7202/MDS7202/main/recursos/2024-01//LLM/agents.png' width=450  />
</center>

## **Manos a la Obra! 👷‍♂️**

<center>
<img src='https://www.yorokobu.es/src/uploads/2014/03/yes.gif' width=350  />
</center>

Ya que conocemos los conceptos básicos de los LLM y LangChain, pasemos ahora a estudiar como implementar estos conceptos en nuestro código. Primero instalamos algunas librerías necesarias:

In [ ]:
%pip install --upgrade --quiet  langchain-google-genai

Luego, pasemos a usar un LLM. Para esta clase utilizaremos `gemini-1.5-flash` de `Google`, pero esto lo pueden cambiar a futuro si disponen de diferentes recursos (por ejemplo, por algún modelo de `OpenAI` o `Azure`):

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash", # modelo de lenguaje
    temperature=0, # probabilidad de "respuestas creativas"
    max_tokens=None, # sin tope de tokens
    timeout=None, # sin timeout
    max_retries=2, # número máximo de intentos
)

llm

Con el modelo ya levantado, podemos interactuar con él de la misma forma que lo hacemos con [ChatGPT](https://chatgpt.com/):

In [ ]:
question = "hola!"
response = llm.invoke(question) # invoke para interactuar con el modelo
response.content

Podemos mejorar la calidad de las respuestas mediante **prompts**, es decir, entregando instrucciones al modelo para que entregue respuestas de alguna manera específica. Para esto, LangChain nos facilita la opción de crear **templates**, los cuales son **instrucciones pre establecidas** que el modelo deberá seguir para responder una pregunta. Por ejemplo:

In [ ]:
from langchain_core.prompts import PromptTemplate

# template: noten como la variable {question} va a ser completada con la pregunta del usuario.
template = '''
Eres un profesional experto en formula 1.
Tu único rol es responder de la forma más completa posible la pregunta del usuario.

Pregunta: {question}
Respuesta útil:
'''

prompt = PromptTemplate.from_template(template)
prompt

Noten como nuestro prompt comparte el método `.invoke`:

In [ ]:
print(prompt.invoke("hola").text)

Esto pasa porque tanto el `ChatModel` como `PromptTemplate` implementan la interfaz `LCEL` (Langchain Expressión Language). Con esto en consideración, podemos juntar el LLM y el prompt creado!

En `langchain` podemos utilizar el operador `|` para **concatenar** diferentes acciones que queremos que nuestra aplicación realice (a esto se le llama **chains**). En este caso, deseamos que la LLM utilice una plantilla para responder. Esto lo hacemos de la siguiente manera:

In [ ]:
chain = prompt | llm # definimos la cadena
response = chain.invoke("hola!") # interactuamos con ella a través de invoke
response

In [ ]:
print(response.content) # print a la respuesta

Noten como la respuesta del LLM es un `AIMessage`, el cual contiene la respuesta y metadata adicional con respecto al mensaje entregado.

Para recuperar sólo el mensaje, podemos usar `StrOutputParser` en la chain:

In [ ]:
from langchain_core.output_parsers import StrOutputParser

chain = prompt | llm | StrOutputParser() # definimos la cadena
response = chain.invoke("quién es el mejor piloto de la formula 1?") # interactuamos con ella a través de invoke
print(response)

Similar al caso anterior, también podemos definir **más de una variable** de entrada en nuestro prompt:

In [ ]:
from langchain_core.prompts import PromptTemplate

template = '''
Eres un profesional experto en formula 1.
Tu único rol es responder de la forma más completa posible la pregunta del usuario.
Además, debes responder con el idioma que se te indique.

Pregunta: {question}
Idioma: {language}
Respuesta útil:
'''

prompt = PromptTemplate.from_template(template)

chain = prompt | llm
response = chain.invoke({"question": "hola!", "language": "chinese"}) # noten como ahora invoke recibe un diccionario
print(response.content)

## Experimento: Conversación filosófica entre 2 LLM 😱

Ya que conocemos lo básico para interactuar con LLMs a través de código, podemos ejecutar algunos experimentos interesantes. Por ejemplo, **qué pasaría si hacemos que 2 LLM conversen sobre filosofía entre sí?**

Probemos!

<center>
<img src='https://tvquotes.co/wp-content/uploads/2017/09/you-pass-butter.gif' width=400/>
</center>



In [ ]:
import time

# instrucciones generales
template = """
Eres un experto en filosofía.

Responde la conversación de manera acorde.
{question}
"""
prompt = PromptTemplate.from_template(template) # prompt

# chains
llm_1 = (prompt
         | llm
         | StrOutputParser()
         )

llm_2 = (prompt
         | llm
         | StrOutputParser()
         )

initial_msg = "me llamo sebastián" # mensaje inicial
msg_1, msg_2 = None, None # mensaje de cada agente

for i in range(10):

    print(f"INTERACCIÓN {i}")

    msg_1 = llm_1.invoke(initial_msg if msg_2 is None else msg_2) # llm_1 recibe msg_2 y genera msg_1
    msg_2 = llm_2.invoke(msg_1) # llm_2 recibe msg_1 y genera msg_2

    # print de mensajes
    print("Agente_1:", msg_1)
    print("Agente_2:", msg_2)
    print('----' * 50)

    time.sleep(5)

Volvamos ahora a la clase.

**Qué pasaría si le pregunto al LLM algúna pregunta de actualidad?** Por ejemplo:

In [ ]:
response = llm.invoke("qué sabes sobre las elecciones municipales 2024 de Chile?")
print(response.content)

**Pregunta**: ¿Porqué podría estar pasando esto? ¿Qué podemos hacer para arreglarlo?

## **Retrieval Augmented Generation (RAG) 🔎📖**

**Retrieval Augmented Generation (RAG)** es una técnica para **ampliar el conocimiento** de los modelos de lenguaje grandes (LLMs) con **datos adicionales**, usualmente provenientes de **fuentes externas**. Si bien podemos hacer RAG con cualquier tipo de información externa, para esta sección nos concentraremos en hacer RAG sobre **documentos PDF**. En otras palabras, estaremos **nutriendo a nuestro LLM** con la información contenida en uno o más **documentos con extensión .pdf**.

<center>
<img src='https://raw.githubusercontent.com/MDS7202/MDS7202/main/recursos/2024-01//LLM/rag-framework.webp' width=450/>
</center>

### **¿Cómo funciona?**

La idea principal de esta solución es **responder la pregunta del usuario usando sólo los fragmentos de texto relevantes** para la pregunta en cuestión.

Con esto en consideración, para implementar una solución RAG sobre documentos PDF debemos pasar por 2 grandes etapas:

**1. Indexing**: Es el proceso por el cual **transformamos nuestros documentos** (.pdf) a **representaciones vectoriales**. Este es un paso que **se realiza sólo una vez**, pero fudamental para recuperar los fragmentos de texto relevantes.

 A su vez, para indexar un documento es necesario pasar por los siguientes pasos:
 - **Load**: **Ingesta del documento** al ambiente de trabajo (e.g: lectura y almacenamiento en strings).
 - **Split**: **Separación del documento en chunks de texto**. Chunks muy grandes pueden diluir la información necesaria para responder, chunks muy chicos pueden inducir a que se pierda el contexto.
 - **Embed**: Cada uno de los **chunks generados se transforman en representaciones vectoriales**, los cuales tienen la capacidad de almacenar la semántica, sintaxis y el contexto del chunk. Para esto, se utilizan modelos de lenguaje especializados en este tipo de transformaciones.
 - **Store**: Con los documentos vectorizados, **se almacenan estas representaciones** en una *vector store*.

<center>
<img src='https://python.langchain.com/assets/images/rag_indexing-8160f90a90a33253d0154659cf7d453f.png' width=700/>
</center>

**2. Retrieval and Generation**: Es el proceso en la que proporcionamos información al LLM para que responda de manera acorde. Consta de las siguientes etapas:
- **Retrieve**: Proceso por el cual **se buscan los *top k* chunks más relevantes a la pregunta**. Para lograr esto, se utilizan técnicas de recuperación de información.
- **Generate**: **Se entregan los *top k*** chunks al LLM y **se responde la pregunta** del usuario.

<center>
<img src='https://python.langchain.com/assets/images/rag_retrieval_generation-1046a4668d6bb08786ef73c56d4f228a.png' width=700/>
</center>

Ya que conocemos el funcionamiento básico de una solución RAG, pasemos a ilustrar esto con un ejemplo. Para esto, construiremos un RAG a partir del  [Informe de las Elecciones Municipales 2024](https://www.decidechile.cl/informes/informe-municipales-2024/) realizado por [Decide Chile](https://www.decidechile.cl/).

Primero instalamos las librerias necesarias:

In [ ]:
%pip install --upgrade --quiet faiss-cpu langchain_community pypdf

### **Indexing**

Recordando los pasos descritos al principio de esta sección, la primera tarea para implementar una solución RAG es **indexar los documentos**.  

#### **Load**

Ya que nuestro documento es de extensión .pdf, comenzaremos **ingestando el documento** usando `PyPDFLoader` (pueden consultar más loaders disponibles en la siguiente [página](https://python.langchain.com/docs/integrations/document_loaders/)):

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

file_path = "https://gitlab.com/sebatinoco/datos/-/raw/main/Informe-DecideChile-Municipales-2024.pdf?inline=false" # path al documento
loader = PyPDFLoader(file_path) # inicializar loader de PDF

docs = loader.load() # cargar documento
docs

In [ ]:
len(docs) # noten como se genera una lista de 55 elementos, en este caso cada elemento es una página

In [ ]:
docs[0] # cada elemento es del tipo Document, el cual posee el contenido de cada página

#### **Split**

Después de haber ingestado el documento, el siguiente paso es dividir cada documento en chunks de texto más pequeños. Para eso usaremos `RecursiveCharacterTextSplitter` (nuevamente, pueden consultar otros tipos de splitters en el siguiente [link](https://python.langchain.com/v0.1/docs/modules/data_connection/document_transformers/)):

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50) # inicializamos splitter
splits = text_splitter.split_documents(docs) # dividir documentos en chunks
splits[:5]

In [ ]:
splits[0] # cada elemento es un Document, esta vez con menos contenido que en el paso anterior

In [ ]:
len(splits) # noten como ahora contamos con 162 chunks (pues 1 página contiene más de 1 chunk)

#### **Embed & Store**

Con los documentos separados en chunks, pasamos ahora a transformarlos a *embeddings*. Para esto simplemente ocupamos embeddings de Google a través de `GoogleGenerativeAIEmbeddings` (de nuevo, pueden revisar más opciones de embeddings en la siguiente [página](https://python.langchain.com/v0.1/docs/integrations/text_embedding/)).

Por el lado del vector store, usaremos `FAISS` (nuevamente, pueden revisar más opciones en este [link](https://python.langchain.com/v0.1/docs/modules/data_connection/vectorstores/)).

In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import FAISS

embedding = GoogleGenerativeAIEmbeddings(model="models/embedding-001") # inicializamos los embeddings
vectorstore = FAISS.from_documents(documents=splits, embedding=embedding) # vectorizacion y almacenamiento
vectorstore

### **Retrieval and Generation**

Con los documentos indexados, el siguiente paso es **habilitar que nuestro LLM pueda conseguir la información de los documentos** para responder preguntas.

#### **Retrieve**

Para esto, el primer paso es habilitar nuestro *vector store* para buscar los documentos más relevantes. Para esto, simplemente usamos el método `.as_retriever`:

In [ ]:
retriever = vectorstore.as_retriever(search_type="similarity", # método de búsqueda
                                     search_kwargs={"k": 3}, # n° documentos a recuperar
                                     )
retriever

Con esto, podemos probar nuestro retriever:

In [ ]:
question = "como fue la participacion en las elecciones?" # pregunta
relevant_documents = retriever.invoke(question) # top k documentos relevantes a la pregunta
relevant_documents

Noten como la información relevante está almaenada en el atributo **page_content**:

In [ ]:
relevant_documents[0].page_content

Como solo nos interesa ese atributo, podemos definir la función `format_docs` para procesar los chunks:

In [ ]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

print(format_docs(relevant_documents))

Finalmente, `LangChain` nos permite unir todos estos pasos en una **chain**:

In [ ]:
retriever_chain = retriever | format_docs # chain
print(retriever_chain.invoke("como fue la participacion en las elecciones?"))

#### **Generation**

Con el retriever ya listo, la única pieza faltante es **entregar los documentos relevantes al LLM** para que pueda responder de mejor manera.

Para esto, primero definiremos un **template** para con las instrucciones a seguir:

In [ ]:
from langchain_core.prompts import PromptTemplate

# noten como ahora existe el parámetro de context!
rag_template = '''
Eres un asistente experto en la interpretación de resultados electorales de la política chilena.
Tu único rol es contestar preguntas del usuario a partir de información relevante que te sea proporcionada.
Responde siempre de la forma más completa posible y usando toda la información entregada.
Responde sólo lo que te pregunten a partir de la información relevante, NUNCA inventes una respuesta.

Información relevante: {context}
Pregunta: {question}
Respuesta útil:
'''

rag_prompt = PromptTemplate.from_template(rag_template)

Finalmente, condensamos todo en una chain para recuperar información relevante y responder al mismo tiempo:

In [ ]:
from langchain_core.runnables import RunnablePassthrough

rag_chain = (
    {
        "context": retriever_chain, # context lo obtendremos del retriever_chain
        "question": RunnablePassthrough(), # question pasará directo hacia el prompt
    }
    | rag_prompt # prompt con las variables question y context
    | llm # llm recibe el prompt y responde
    | StrOutputParser() # recuperamos sólo la respuesta
)

question = "que información tienes sobre nulos y blancos?"
response = rag_chain.invoke(question)
print(response)

## **Agentes 🕵️‍♂️**

Ahora que ya conocemos cómo implementar un RAG básico sobre documentos PDF, buscaremos hacer lo mismo (es decir, nutrir nuestro LLM de fuentes externas) pero con dos diferencias:

- Alimentaremos nuestro agente de la información proporcionada por **motores de búsqueda** (e.g: Google).
- Remplazaremos las chains por **Agentes**.

Todo genial, pero...

### **¿Qué son los Agentes?**

Similar a lo que vimos en la clase de RL, un agente se define como un modelo que tiene la capacidad para ejecutar **acciones**. De esta manera, el objetivo del agente es **elegir una <u>secuencia</u> de acciones** a realizar para cumplir con un objetivo específico.

> **Pregunta**: Ok, pero entonces cual es la diferencia con las chains?

Si bien las **chains** tienen la capacidad de implementar una secuencia de acciones o pasos, estas acciones estan **programadas de manera fija** en el código. En contraste, los **agentes** utilizan el **LLM como <u>motor de razonamiento</u>** para determinar **qué acciones tomar y en qué orden**.

Ahora que ya conocemos qué es un Agente, conozcamos uno de los framework más famosos para implementar Agentes con LLM.

<center>
<img src='https://media4.giphy.com/media/dBZsIa2eWkVBaU6lWY/giphy.gif' width=450/>
</center>

### **ReAct**

Introducido en el paper de [Yao et al. (2022)](https://arxiv.org/abs/2210.03629), **ReAct** se presenta como un framework para que agentes puedan **razonar sobre acciones** a partir de observaciones. En particular, para cumplir un objetivo un agente basado en ReAct debe seguir la siguiente secuencia:

- El agente **razona** sobre qué acción tomar.
- En base al razonamiento hecho, el **agente ejecuta la acción**.
- A partir de la acción ejecutada, el **agente observa y evalúa el nuevo escenario** (feedback).

> **Pregunta**: ¿Qué parecido encuentran con lo que hemos visto hasta ahora?

<center>
<img src='https://peterroelants.github.io/images/llm/ReAct_loop.png' width=400/>
</center>

Por último, es importante señalar que Los resultados muestran como los **agentes basados en ReAct** puede **superar múltiples benchmarks en tareas de lenguaje y toma de decisiones**, además de mejorar la interpretabiliad y confiabilidad en los LLM.

Pongamos en práctica lo aprendido con un ejemplo!

Comencemos primero cargando un prompt predefinido del **hub** de langchain para usar ReAct:

In [ ]:
from langchain import hub

react_prompt = hub.pull("hwchase17/react") # template de ReAct
print(react_prompt.template)

> **Pregunta:** Tómense un momento para estudiar el prompt. ¿Qué variables recibe?

Una de las ventajas de usar agentes es que tienen una fácil integración con **tools**, es decir, **herramientas que puede usar el agente para lograr un objetivo** en particular.

Para este caso particular, usaremos la tool del motor de búsqueda `Tavily` para permitir que nuestro agente pueda recuperar **información de la web** (pueden consultar más tools en el siguiente [link](https://api.python.langchain.com/en/v0.1/community_api_reference.html#module-langchain_community.tools)):

In [ ]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults(max_results = 1) # inicializamos tool
tools = [search] # guardamos las tools en una lista

Con las tools definidas, podemos inicializar nuestro agente ReAct:

In [ ]:
from langchain.agents import create_react_agent, AgentExecutor

agent = create_react_agent(llm, tools, react_prompt) # primero inicializamos el agente ReAct
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True) # lo transformamos a AgentExecutor para habilitar la ejecución de tools
agent_executor

Finalmente, probamos nuestro agente:

In [ ]:
response = agent_executor.invoke({"input": "qué equipo ganó el mundial de LoL 2024?"})
print(response["output"])

### **Implementando nuestras propias tools**

Algo interesante que podemos hacer es **programar nuestras propias tools** para que el agente interactúe con ellas.

Revisemos un ejemplo en que programos tools con algunas **operaciones matemáticas**:

<center>
<img src='https://media1.tenor.com/images/dfe0c1c6eaf41b91996aacee0879ebc2/tenor.gif?itemid=3486402' width=400  />
</center>

In [ ]:
from langchain.tools import tool

@tool
def multiply(x: int or float, y: int or float) -> float:
    """Multiply 'x' times 'y'."""
    return float(x * y)

@tool
def exponentiate(x: int or float, y: int or float) -> float:
    """Raise 'x' to the 'y'."""
    return float(x**y)

@tool
def add(x: int or float, y: int or float) -> float:
    """Add 'x' and 'y'."""
    return float(x + y)

Luego, simplemente agrupamos las tools en una lista:

In [ ]:
tools = [add, multiply, exponentiate]

En paralelo, crearemos un **prompt** para nuestro agente:

In [ ]:
# noten como ahora se incluye la variable agent_scratchpad
math_template = """
Eres un asistente experto en matemáticas.
Tu único rol es responder la pregunta del usuario usando las tools disponibles.

Pregunta: {input}
{agent_scratchpad}
"""

prompt = PromptTemplate.from_template(math_template)
prompt

Con el prompt creado, pasamos a **crear nuestro agente**.

Noten que como nuestras tools reciben más de un parámetro de entrada (a y b), **remplazaremos ReAct por [Tool Calling](https://python.langchain.com/v0.1/docs/modules/agents/agent_types/tool_calling/)** (de igual manera, pueden encontrar todos los tipos de Agentes disponibles y sus limitantes en el siguiente [link](https://python.langchain.com/v0.1/docs/modules/agents/agent_types/)):

In [ ]:
from langchain.agents import create_tool_calling_agent

agent = create_tool_calling_agent(llm, tools, prompt)
math_agent = AgentExecutor(agent=agent, tools=tools, verbose=True)
math_agent

Finalmente, podemos probar el funcionamiento de nuestro agente:

In [ ]:
math_agent.invoke({"input": "cuanto es 10 ** 3 + 5 * 1.4?",})

## **Soluciones Multi Agente 👨‍👩‍👦‍👦**

<center>
<img src='https://media.tenor.com/FApRE_u99tgAAAAC/teamwork-team-game.gif' width=400  />
</center>

En las secciones pasadas habilitamos agentes que puedan hacer RAG sobre fuentes externas:

- Una **chain** que responde preguntas de las últimas elecciones municipales en base a un informe en PDF
- Un **agente** que responde preguntas matemáticas a partir de tools creadas manualmente.

Con esto en consideración, nace la pregunta natural: **¿Qué pasa si combinamos ambas soluciones?**

El objetivo de esta sección es introducirlos al paradigma **multiagente**, es decir, **combinar 2 o más funcionalidades en un mismo chat**. En particular, buscaremos implementar una arquitectura simple de enrutamiento, la cual consta de 4 agentes:

- **Agente router**, el cual recibe y dirige la pregunta del usuario a alguno de los agentes.
- **Agente de elecciones**: responde preguntas sobre las elecciones municipales 2024
- **Agente experto en matemáticas**: responde preguntas matemáticas
- **Agente de redireccionamiento**: en caso de que la pregunta del usuario no pertenezca a alguno de los temas anteriores, invita al usuario a reorientar su pregunta (esto es útil para evitar preguntas maliciosas).

**Nota**: Para efectos de esta sección y por simplicidad, no se hace distinción entre Agente y Chain.

<center>
<img src='https://preview.redd.it/smart-orchestrator-router-for-multiple-specialized-llms-v0-gjgkmlbu3jlc1.png?width=627&format=png&auto=webp&s=13ef701d45f642ce36ae8e99cb172b903fe7d36b' width=600  />
</center>

**Importante: La implementación es sólo ilustrativa y <u>no cumple con el estándar actual recomendado por LangChain</u>. Si desean conocer mejor cómo implementar soluciones multiagente, les recomiendo estudiar [LangGraph](https://www.langchain.com/langgraph)**.

#### **Agente Router**

Primero comenzamos creando nuestro agente router:

In [ ]:
router_prompt = PromptTemplate.from_template(
    """
    Eres un asistente experto en la clasificación de preguntas del usuario.
    Tu único rol es clasificar preguntas del usuario en las categorías 'elecciones', 'math', u 'otro' según el siguiente criterio:
    - 'elecciones': Cuando la pregunta sea relacionada con las elecciones municipales de chile 2024.
    - 'math': Cuando la pregunta sea relacionada a preguntas de matemáticas
    - 'otro': Todo aquella pregunta que no esté contenida en las categorías anteriores.

    No respondas con más de una palabra y no incluyas.

    <pregunta>
    {question}
    </pregunta>

    Categoría:"""
)

router_chain = (
    router_prompt
    | llm
    | StrOutputParser()
)

router_chain.invoke({"question": "cuanto es 2+2"})

#### **Agente Redirect**

Repetimos lo mismo para crear nuestro agente de redireccionamiento:

In [ ]:
redirect_prompt = PromptTemplate.from_template(
    """
    Eres un asistente experto en el redireccionamiento de preguntas de usuarios.
    Vas a recibir una pregunta del usuario, tu único rol es indicar que no puedes responder su pregunta y redireccionar al usuario
    para que te pregunte sobre las elecciones municipales de Chile 2024 o cálculos matemáticos.

    Recuerda ser amable y cordial en tu respuesta.

    Pregunta: {question}
    Respuesta cordial:"""
)

redirect_chain = (
    redirect_prompt
    | llm
    | StrOutputParser()
)

redirect_chain.invoke({"question": "dame la receta para hacer una pizza"})

#### **Juntando todo**

Finalmente, podemos juntar todo lo que hemos desarrollado en una sola función:

In [ ]:
def route_question(question):
  '''
  Recibe una pregunta de usuario.
  Rutea la pregunta al agente respectivo y responde de manera acorde.
  '''

  topic = router_chain.invoke({"question": question}) # enrutamiento

  if "elecciones" in topic: # si la pregunta es sobre las elecciones, utilizar cadena
      return rag_chain.invoke(question)
  elif "math" in topic: # si la pregunta es de matemáticas, utilizar agente
      return math_agent.invoke({"input": question})["output"]
  else: # de lo contrario, redireccionar pregunta
      return redirect_chain.invoke({"question": question})

Para finalmente hacer pruebas de su funcionamiento:

In [ ]:
:print(route_question("cómo puedo hacerme millonario?"))

In [ ]:
print(route_question("como fue la participación de las elecciones?"))

In [ ]:
print(route_question("cuanto es 5+3?"))